In [ ]:
# Cell 1: Setup environment and imports
import logging
import os
import time
from datetime import datetime
from dataclasses import dataclass, field
from typing import Tuple, List
import random
import json
import pygame
from llama_cpp import Llama
import kagglehub

COLORS = ["white", "black"]
FIGURE_TYPES = ["king", "queen", "rook", "bishop", "knight", "pawn"]

class TimestampedLogger:
    def __init__(self, log_dir='logs', log_file='simulation.log'):
        os.makedirs(log_dir, exist_ok=True)
        self.log_path = os.path.join(log_dir, log_file)
        logging.basicConfig(filename=self.log_path, level=logging.INFO, filemode='w')
        self.start_time = time.time()
        self.last_time = self.start_time
        self.log("Logger initialized.")

    def _now(self):
        return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    def _duration(self):
        current_time = time.time()
        duration = current_time - self.last_time
        self.last_time = current_time
        return f"{duration:.3f}s"

    def log(self, message):
        timestamp = self._now()
        duration = self._duration()
        log_message = f"[{timestamp}] (+{duration}) {message}"
        print(log_message)
        logging.info(log_message)
LOGGER = TimestampedLogger()

def load_config(config_path: str = "config.json") -> dict:
    """Loads the configuration file."""
    LOGGER.log(f"Load Config: {config_path}")
    with open(config_path, "r") as f:
        return json.load(f)
CONFIG = load_config("config.json")

class _Figure:
    """Represents a figure on the board."""
    def __init__(self, position: Tuple[int, int], color: str, figure_type: str):
        self.position = position
        self.color = color
        self.figure_type = figure_type

        self.target_positions = []  # List of target positions this figure can attack or defend

    def calculate_figure_targets(self, board: List[List['_Tile']]):
        """Generates a list of target positions based on the figure's movement rules."""
        # Calculate movement rules based on figure type
        directions = []
        max_board_size = max(CONFIG["board"]["width"], CONFIG["board"]["height"])
        max_range = max_board_size if self.figure_type in ["queen", "rook", "bishop"] else 1
        if self.figure_type == "king" or self.figure_type == "queen":
            directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]
        elif self.figure_type == "rook":
            directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        elif self.figure_type == "bishop":
            directions = [(1, 1), (-1, -1), (1, -1), (-1, 1)]
        elif self.figure_type == "knight":
            directions = [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2)]
        elif self.figure_type == "pawn":
            # Pawns can only capture diagonally forward
            if self.color == "white":
                directions = [(1, 1), (-1, 1)]
            else:
                directions = [(1, -1), (-1, -1)]

        # Generate valid target positions based on its movement rules
        for direction in directions:
            for dr in range(1, max_range + 1):
                dx = direction[0] * dr
                dy = direction[1] * dr
                target_position = (self.position[0] + dx, self.position[1] + dy)
                if      0 <= target_position[0] < CONFIG["board"]["width"] \
                    and 0 <= target_position[1] < CONFIG["board"]["height"]:
                    self.target_positions.append(target_position)
                    print(f"Figure {self.color} {self.figure_type} at {self.position} can target {target_position}.")
                    if board[target_position[0]][target_position[1]].figure is not None:
                        # Stop if we hit another figure
                        break

class _Drone:
    """Represents a drone in the simulation."""
    def __init__(self, id: int, position: Tuple[int, int], model_path: str, rules: str, color: str = "white"):
        self.id = id
        self.position = position
        self.color = color
        self.model_path = model_path
        self.memory = [{"rules": rules}]  # Memory of past actions or observations

    def generate_model_response(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1280) -> str:
        """
        Generate a response from a Qwen GGUF model using llama-cpp.

        Args:
            prompt (str): The input prompt to the language model.
            temperature (float): Sampling temperature.
            max_tokens (int): Maximum tokens to generate in the response.

        Returns:
            str: The model's text response.
        """
        llm = Llama(
            model_path=self.model_path,
            n_ctx=2048,
            temperature=temperature,
            n_threads=os.cpu_count() // 2  # Adjust to your CPU
        )

        response = llm(prompt, max_tokens=max_tokens, stop=["</s>"], echo=False)
        # return response["choices"][0]["text"].strip()
        return str(response)

    def get_action(self):
        """Determines the action for the drone based on its rules."""
        # Placeholder for action logic: move, wait, or broadcast
        LOGGER.log(f"Drone {self.id} at position {self.position} is taking action.")
        model_input = self.memory[-1]  # Use the last memory entry as input
        print(self.generate_model_response(f"Drone {self.id} at position {self.position} with memory {model_input} decides its next action."))

class _Tile:
    """Represents a tile on the board."""
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y
        self.targeted_by = {"white": 0, "black": 0}
        self.figure = None
        self.drones = []

    def set_figure(self, figure: _Figure):
        """Sets a piece on the tile."""
        self.figure = figure

    def add_drone(self, drone: _Drone):
        """Adds a drone to the tile."""
        assert drone not in self.drones
        self.drones.append(drone)

    def remove_drone(self, drone: _Drone):
        """Removes a drone from the tile."""
        if drone in self.drones:
            self.drones.remove(drone)

    def reset_targeted_by_amounts(self):
        """Resets the threat and defend levels."""
        self.targeted_by = {"white": 0, "black": 0}

    def add_targeted_by_amount(self, color: str, amount: int = 1):
        """Adds to the targeted by amount for a color."""
        self.targeted_by[color] += amount

class _SimulationGUI:
    def __init__(self):
        """Initializes the GUI for the simulation."""
        self.grid_size = SIM.grid_size
        
        # Initialize Pygame
        pygame.init()
        gui_config = CONFIG["gui"]
        self.screen = pygame.display.set_mode(
            (self.grid_size[0] * (gui_config["cell_size"] + gui_config["margin"]) + gui_config["margin"],
             self.grid_size[1] * (gui_config["cell_size"] + gui_config["margin"]) + gui_config["margin"])
        )
        pygame.display.set_caption(f"Simulation - Round {1}.1/{SIM.max_rounds}.{SIM.num_drones}")
        self.clock = pygame.time.Clock()
        
    def draw_field(self):
        """Draws the board, king, and drones using the current state."""
        gui_config = CONFIG["gui"]
        cell_size = gui_config["cell_size"]
        margin = gui_config["margin"]
        grid_width, grid_height = self.grid_size

        self.screen.fill(gui_config["background_color"])
        font = pygame.font.SysFont(None, 16)

        for x in range(grid_width):
            for y in range(grid_height):
                rect = pygame.Rect(
                    x * (cell_size + margin) + margin,
                    y * (cell_size + margin) + margin,
                    cell_size,
                    cell_size
                )
                pygame.draw.rect(self.screen, gui_config["grid_color"], rect)

                tile = SIM.board[x][y]

                # Draw King
                if tile.figure and tile.figure.figure_type == "king":
                    pygame.draw.circle(self.screen, gui_config["king_color"], rect.center, cell_size // 3)

                # Draw drones (multiple per tile supported)
                total = len(tile.drones)
                if total > 0:
                    angle_step = 360 / total if total > 1 else 0
                    radius = cell_size // 6

                    for idx in range(len(tile.drones)):
                        angle = angle_step * idx
                        offset = pygame.math.Vector2(0, 0)
                        if total > 1:
                            offset = pygame.math.Vector2(1, 0).rotate(angle) * (cell_size // 4)
                        center = (rect.centerx + int(offset.x), rect.centery + int(offset.y))

                        pygame.draw.circle(self.screen, gui_config["drone_color"], center, radius)
                        text = font.render(str(idx + 1), True, gui_config["text_color"])
                        text_rect = text.get_rect(center=center)
                        self.screen.blit(text, text_rect)
                        
        pygame.display.flip()

class Simulation:
    """Tracks the state of the simulation."""
    def __init__(self):
        # Initialize game state
        self.turn = 1 # Which drone's turn it is
        self.round = 1 # Which round of the game it is
        self.rules = ""        
        with open(CONFIG["simulation"]["rules_path"], "r") as f:
            self.rules = f.read().replace("NUMBER_OF_DRONES", str(CONFIG["simulation"]["num_drones"]))
        self.grid_size = (CONFIG["board"]["width"], CONFIG["board"]["height"])
        self.max_rounds = CONFIG["simulation"]["max_rounds"]
        self.num_drones = CONFIG["simulation"]["num_drones"]
        self.model_path = os.path.join(kagglehub.model_download(CONFIG["simulation"]["model"]), CONFIG["simulation"]["model_name"])
        print(f"Model downloaded to {self.model_path}")

        # Create the board
        self.board = [[_Tile(x, y) for y in range(self.grid_size[1])] for x in range(self.grid_size[0])]
        print(len(self.board), "x", len(self.board[0]), "board created.")
        self.figures = []  # List of all figures on the board
        self.drones = []  # List of drones in the simulation

        # Create, place and calculate figures
        self._create_figures()

        # Create and place drones
        self.drone_base = self.figures[0].position # Assuming the first figure is the drone base (white king)
        self._create_drones()

        # Initialize the GUI
        self.gui = _SimulationGUI()

    def _create_figures(self):
        """Generates and places all figures based on CONFIG["figures"]."""
        # Create figures based on the configuration
        LOGGER.log("Creating figures based on configuration.")
        self.figures = []
        for color in COLORS:
            for figure_type in FIGURE_TYPES:
                for position in CONFIG["figures"][color][figure_type]:
                    self.figures.append(_Figure(position, color, figure_type))

        # Place figures on the board
        for figure in self.figures:
            self.board[figure.position[0]][figure.position[1]].set_figure(figure)
            print(f"Placed {figure.color} {figure.figure_type} at position {figure.position}.")
        
        # Calculate targets for each figure
        for figure in self.figures:
            figure.calculate_figure_targets(self.board)

        # Add targeted by amounts to the tiles based on figure targets
        for figure in self.figures:
            for target_position in figure.target_positions:
                target_tile = self.board[target_position[0]][target_position[1]]
                target_tile.add_targeted_by_amount(figure.color, 1)

    def _create_drones(self):
        """Generate drones."""
        LOGGER.log(f"Creating {self.num_drones} drones.")
        for i in range(self.num_drones):
            drone = _Drone(id=i+1, position=self.drone_base, model_path=self.model_path, rules=self.rules)
            self.drones.append(drone)
            
        # Place drones on the board
        for drone in self.drones:
            tile = self.board[drone.position[0]][drone.position[1]]
            tile.add_drone(drone)

SIM = Simulation()

[2025-07-18 00:22:26] (+0.000s) Logger initialized.
[2025-07-18 00:22:26] (+0.000s) Load Config: config.json
Model downloaded to C:\Users\CKamm\.cache\kagglehub\models\qwen-lm\qwen-3\gguf\4b\1\Qwen3-4B-Q8_0.gguf
8 x 8 board created.
[2025-07-18 00:22:26] (+0.530s) Creating figures based on configuration.
Placed white king at position [4, 0].
Placed white queen at position [3, 0].
Placed white rook at position [0, 0].
Placed white bishop at position [2, 0].
Placed white bishop at position [5, 0].
Placed white knight at position [1, 0].
Placed white knight at position [6, 0].
Placed white pawn at position [0, 1].
Placed white pawn at position [1, 1].
Placed white pawn at position [2, 1].
Placed white pawn at position [3, 1].
Placed white pawn at position [4, 1].
Placed white pawn at position [5, 1].
Placed white pawn at position [6, 1].
Placed white pawn at position [7, 1].
Placed black king at position [4, 7].
Placed black queen at position [3, 7].
Placed black rook at position [0, 7].


In [52]:
# Cell 2: Define real-time simulation GUI using Pygame
def run_simulation(sim: Simulation):
    """Runs the simulation with real-time GUI display using Pygame."""

    max_rounds = CONFIG["simulation"].get("max_rounds", 10)
    delay = CONFIG["simulation"].get("delay", 300)

    directions = [(-1, -1), (-1, 0), (-1, 1),
                  ( 0, -1), ( 0, 0), ( 0, 1),
                  ( 1, -1), ( 1, 0), ( 1, 1)]

    running = True
    pygame.display.set_caption(f"Simulation - Round {1}.1/{max_rounds}.{SIM.num_drones}")

    for sim_round in range(max_rounds):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                break
        if not running:
            break

        SIM.round = sim_round + 1

        # Move each drone
        for drone in sim.drones:
            SIM.turn = drone.id
            pygame.display.set_caption(f"Simulation - Round {sim_round + 1}.{drone.id}/{max_rounds}.{SIM.num_drones}")
            
            x, y = drone.position
            old_tile = sim.board[x][y]

            # Determine figures on the current tile
            if old_tile.figure:
                LOGGER.log(f"Drone {drone.id} is on tile ({x}, {y}) with figure: {old_tile.figure.color} {old_tile.figure.figure_type}")

            # Determine drones on the current tile
            if old_tile.drones:
                LOGGER.log(f"Drone {drone.id} is on tile ({x}, {y}) with drones: {', '.join(str(drone.id) for drone in old_tile.drones)}")

            drone.get_action()  # Get action from the drone's model
            

            # Remove drone from current tile before movement
            if drone in old_tile.drones:
                old_tile.drones.remove(drone)

            # Determine valid moves
            valid_moves = []
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < SIM.grid_size[0] and 0 <= ny < SIM.grid_size[1]:
                    valid_moves.append((nx, ny))

            # Move drone
            if valid_moves:
                new_pos = random.choice(valid_moves)
                drone.position = new_pos
                new_tile = SIM.board[new_pos[0]][new_pos[1]]
                new_tile.drones.append(drone)
            else:
                # If no valid moves, re-add to current tile
                old_tile.drones.append(drone)
            
            SIM.gui.draw_field()
            pygame.time.delay(delay)
        
            LOGGER.log(f"Round {SIM.round}.{SIM.turn} completed.")



    LOGGER.log("Simulation ended.")
    pygame.quit()


In [53]:
# Cell 4: Optional launcher for real-time simulation with GUI in notebooks

RUN_SIMULATION = True  # Set to False to disable auto-execution in notebooks

if __name__ == "__main__" or RUN_SIMULATION:
    LOGGER.log("Launching real-time simulation with GUI...")
    run_simulation(SIM)


llama_model_loader: loaded meta data with 28 key-value pairs and 398 tensors from C:\Users\CKamm\.cache\kagglehub\models\qwen-lm\qwen-3\gguf\4b\1\Qwen3-4B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen3 4B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Qwen3
llama_model_loader: - kv   5:                         general.size_label str              = 4B
llama_model_loader: - kv   6:                          qwen3.block_count u32              = 36
llama_model_loa

[2025-07-18 00:22:27] (+1.132s) Launching real-time simulation with GUI...
[2025-07-18 00:22:27] (+0.002s) Drone 1 is on tile (4, 0) with figure: white king
[2025-07-18 00:22:27] (+0.000s) Drone 1 is on tile (4, 0) with drones: 1
[2025-07-18 00:22:27] (+0.000s) Drone 1 at position [4, 0] is taking action.


init_tokenizer: initializing tokenizer for type 2
load: control token: 151661 '<|fim_suffix|>' is not marked as EOG
load: control token: 151649 '<|box_end|>' is not marked as EOG
load: control token: 151647 '<|object_ref_end|>' is not marked as EOG
load: control token: 151654 '<|vision_pad|>' is not marked as EOG
load: control token: 151659 '<|fim_prefix|>' is not marked as EOG
load: control token: 151648 '<|box_start|>' is not marked as EOG
load: control token: 151644 '<|im_start|>' is not marked as EOG
load: control token: 151646 '<|object_ref_start|>' is not marked as EOG
load: control token: 151650 '<|quad_start|>' is not marked as EOG
load: control token: 151651 '<|quad_end|>' is not marked as EOG
load: control token: 151652 '<|vision_start|>' is not marked as EOG
load: control token: 151653 '<|vision_end|>' is not marked as EOG
load: control token: 151655 '<|image_pad|>' is not marked as EOG
load: control token: 151656 '<|video_pad|>' is not marked as EOG
load: control token: 151

{'id': 'cmpl-036bd543-c59c-4d16-9536-8f47ecf89fc5', 'object': 'text_completion', 'created': 1752790948, 'model': 'C:\\Users\\CKamm\\.cache\\kagglehub\\models\\qwen-lm\\qwen-3\\gguf\\4b\\1\\Qwen3-4B-Q8_0.gguf', 'choices': [{'text': " Drone 1 is at [4, 0] on a chessboard. The chessboard is 8x8. The drone can move to neighboring squares, which are adjacent horizontally, vertically, or diagonally. So, the possible moves from [4,0] are:\n\n[3,0], [3,1], [4,1], [5,0], [5,1], [4,-1] (but [4,-1] is invalid as it's off the board).\n\nSo the possible moves are:\n\n[3,0], [3,1], [4,1], [5,0", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 430, 'completion_tokens': 128, 'total_tokens': 558}}
[2025-07-18 00:23:15] (+47.634s) Round 1.1 completed.
[2025-07-18 00:23:15] (+0.000s) Simulation ended.
